In [1]:
!pip install minerl anyrl

     |████████████████████████████████| 36.3MB 1.2MB/s 
     |████████████████████████████████| 788kB 57.5MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 5.5MB 23.6MB/s 
     |████████████████████████████████| 460kB 35.8MB/s 
     |████████████████████████████████| 92kB 10.1MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 13.0MB 21.8MB/s 
     |████████████████████████████████| 358kB 38.3MB/s 
     |████████████████████████████████| 92kB 11.5MB/s 
  Created wheel for minerl: filename=minerl-0.3.6-cp36-none-any.whl size=72040787 sha256=03b6223c78ca1f08a4c51136d2eee4e3eaf5ddc638d6a0b36923d57d2ef0dead
  Stored in directory: /root/.cache/pip/wheels/8e/dc/c9/a7f3c17ccc05df396620a9e6921936d3bd35df1f25466c9fd7
  Created wheel for typing: filename=typing-3.7.4.3-cp36-none-any.whl size=26307 sha256=f01068a71bd918f51e777fab3d8016a146459d544af27db263c4a24d33fe922a
  Stored in directory: /root/.c

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import json
import gym

from collections import deque

import numpy as np

from anyrl.rollouts import replay

In [2]:
replay_buffer = replay.PrioritizedReplayBuffer(500000, alpha=0.4, beta=0.6, epsilon=0.001)
file_path = "/content/drive/My Drive/minerl-resources/datasets/parsed_data.sav"
combos_path = "/content/drive/My Drive/minerl-resources/combos.sav"

In [3]:
def load_data(path):
  with open(path, 'rb') as handle:
    return pickle.load(handle)

In [4]:
import pickle

In [13]:
combos = load_data(combos_path)

In [18]:
combos

[(0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 0, 0, 1, 0, 1),
 (0, 0, 0, 0, 0, 0, 1, 1, 0),
 (0, 0, 0, 0, 0, 0, 1, 1, 1),
 (0, 0, 0, 0, 0, 1, 0, 0, 0),
 (0, 0, 0, 0, 0, 1, 0, 0, 1),
 (0, 0, 0, 0, 0, 1, 0, 1, 0),
 (0, 0, 0, 0, 0, 1, 0, 1, 1),
 (0, 0, 0, 0, 0, 1, 1, 0, 0),
 (0, 0, 0, 0, 0, 1, 1, 0, 1),
 (0, 0, 0, 0, 0, 1, 1, 1, 0),
 (0, 0, 0, 0, 0, 1, 1, 1, 1),
 (0, 0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, 1),
 (0, 0, 0, 0, 1, 0, 0, 1, 0),
 (0, 0, 0, 0, 1, 0, 0, 1, 1),
 (0, 0, 0, 0, 1, 0, 1, 0, 0),
 (0, 0, 0, 0, 1, 0, 1, 0, 1),
 (0, 0, 0, 0, 1, 0, 1, 1, 0),
 (0, 0, 0, 0, 1, 0, 1, 1, 1),
 (0, 0, 0, 0, 1, 1, 0, 0, 0),
 (0, 0, 0, 0, 1, 1, 0, 0, 1),
 (0, 0, 0, 0, 1, 1, 0, 1, 0),
 (0, 0, 0, 0, 1, 1, 0, 1, 1),
 (0, 0, 0, 0, 1, 1, 1, 0, 0),
 (0, 0, 0, 0, 1, 1, 1, 0, 1),
 (0, 0, 0, 0, 1, 1, 1, 1, 0),
 (0, 0, 0, 0, 1, 1, 1, 1, 1),
 (0, 0, 0, 1, 0, 0, 0, 0, 0),
 (0, 0, 0,

In [14]:
def save_data(file_name, obj):
  print("Saving processed data...")
  with open(file_name, 'w') as f:
      json.dump(obj, f)

In [5]:
def parse_load_data(data):
  states = []

  for d in data:
    sequence_length = len(d['actions'])
    for i in range(0, sequence_length):
      states.append({
          'done':d['dones'][i],
          'action':d['actions'][i],
          'reward':d['rewards'][i],
          'curr_obs':{
              'obs':d['curr_states']['pov'][i],
              'compassAngle':d['curr_states']['compassAngle'][i],
              'inventory':{'dirt':d['curr_states']['inventory']['dirt'][i]}
          },
          'next_state':{
              'obs':d['next_states']['pov'][i],
              'compassAngle':d['next_states']['compassAngle'][i],
              'inventory':{'dirt':d['next_states']['inventory']['dirt'][i]}
          }
      })
  
  return states

In [7]:
def populate_buffer(data_path, replay_buffer):
  data = load_data(data_path)
  states = parse_load_data(data)
  # save_data("processed_states.json", states)
  print("Saved processed data!")
  print("Populating buffer...")
  states_length = len(states)

  curr_state_deque = deque()
  next_state_deque = deque()
  dones_deque = deque()
  frames_deque = deque()
  actions_deque = deque()
 
  rep_buffer = []

  rewards = []

  parse_ts = 0
  episode_start_ts = 0
  curr_obs = states[0]['curr_obs']['obs']
  last_state_was_done = False

  for i in range(1, states_length):
    episode_start_ts += 1
    parse_ts += 1

    next_obs = states[i]['curr_obs']['obs']
    reward = states[i]['reward']
    _a = parse_actions(states[i]['action'])
    #_a = match_actions(_a, combos)
    done = parse_done(states[i]['done'])

    reward = np.sign(reward) * np.log(1.+np.abs(reward))
 
    curr_state_deque.append(curr_obs)
    actions_deque.append(_a)
    rewards.append(reward)
    next_state_deque.append(next_obs)
    dones_deque.append(done)

    print(actions_deque)
    if episode_start_ts > 10:
      add_transition(replay_buffer, curr_state_deque, actions_deque, rewards, next_state_deque, dones_deque, curr_obs)
    if done == 'True':
      add_transition(replay_buffer, curr_state_deque, actions_deque, rewards, next_state_deque, dones_deque, curr_obs)
      curr_obs = states[(i + 1)]['curr_obs']['obs']
      i = i + 2

      curr_state_deque.clear()
      actions_deque.clear()
      rewards.clear()
      next_state_deque.clear()
      dones_deque.clear()

      episode_start_ts = 0
    else:
      curr_obs = next_obs

  add_transition(replay_buffer, curr_state_deque, actions_deque, rewards, next_state_deque, dones_deque, curr_obs)

  return replay_buffer

In [8]:
def add_transition(replay_buffer, curr_states, actions, rewards, next_steps, dones, curr_state, empty_deque=False, ns=10, ns_gamma=0.99,is_done=True):
  ns_rew_sum = 0.
  trans = {}

  if empty_deque:
    while len(rewards) > 0:
      for i in range(len(rewards)):
        ns_rew_sum += rewards[i] * ns_gamma ** i
      
      trans['sample'] = [curr_states.popleft(), actions.popleft(), rewards.pop(0),
                                      next_steps.popleft(), is_done, ns_rew_sum, curr_state]
      replay_buffer.add_sample(trans)
  else:
    for i in range(ns):
      ns_rew_sum += rewards[i] * ns_gamma ** i
    trans['sample'] =  [curr_states.popleft(), actions.popleft(), rewards.pop(0),
                           next_steps.popleft(), dones.popleft(), ns_rew_sum, curr_state]
    replay_buffer.add_sample(trans)

In [12]:
replay_buffer = populate_buffer(file_path, replay_buffer)

Saved processed data!
Populating buffer...
deque([[0, 0, 1, 0, 0, 0, 0, 0, 0]])
deque([[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]])
deque([[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]])
deque([[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]])
deque([[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]])
deque([[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]])
deque([[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0]])
deque([[0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0,

In [109]:
import pickle

In [13]:
filename = 'buffer_actions.sav'
pickle.dump(replay_buffer, open("/content/drive/My Drive/minerl-resources/"+filename, 'wb'))

In [19]:
data = load_data(file_path)

In [106]:
len(combos)

512

In [63]:
a = parse_actions(data[0]['actions'][31])

In [60]:
data[0]['actions'][31]

OrderedDict([('attack', 0),
             ('back', 0),
             ('camera', [0.0, 0.0]),
             ('forward', 1),
             ('jump', 0),
             ('left', 1),
             ('place', 0),
             ('right', 0),
             ('sneak', 0),
             ('sprint', 0)])

In [40]:
list(combos[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0]

In [41]:
type(a)

list

In [92]:
c = match_actions(a, combos)

In [93]:
c

80

In [91]:
def match_actions(action, combos):
  return [i for i in range(0, len(combos)) if list(combos[i]) == a][0]

In [10]:
def parse_actions(actions):
  keys = actions.keys()
  
  parsed = []

  for key in keys:
    if key != 'camera':
    #   parsed.append(actions[key][0])
    #   parsed.append(actions[key][1])
    # else:
      parsed.append(actions[key])
  return parsed

In [11]:
def parse_done(done):
  if done == 'True':
    return 1
  else:
    return 0